<a href="https://www.kaggle.com/code/adamdandi/2025-training-performance-analysis?scriptVersionId=269105975" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **2025 Training Performance Analysis: Session Delivery Overview**

## **Introduction**

This document provides an overview of the training performance across 2025. It aims to analyse the number of sessions delivered during the year to understand the overall demand for each course. The analysis supports data-driven planning for future training programs and helps identify areas with higher participation or emerging needs. This aligns with the organisation’s goal to improve training effectiveness and resource allocation.

## **Objectives**

The key objectives of this documentation are to:

* Determine the total number of sessions delivered per course during 2025.
* Identify trends in session frequency across different training categories.
* Highlight areas of growth or decline to guide future program planning.
* Support decisions on course scheduling, instructor allocation, and resource investment.

Success will be measured by the clarity and accuracy of insights drawn from the data, enabling actionable planning for the next training cycle.

## **Scope**

The analysis covers all training sessions delivered between January and December 2025. It includes course titles, frequency, and participation data where available. The scope is limited to quantitative delivery metrics and does not include qualitative feedback or performance evaluation. All data is sourced from the internal training records and validated for consistency.

## **Methodology**

The analysis follows these key steps:

1. Collect and verify session data from the 2025 training records.
2. Organise courses by category and delivery frequency.
3. Apply filters to identify patterns and variations in demand.
4. Summarise findings in tables and charts for clarity.

This structured approach ensures that the data is complete, accurate, and ready for interpretation.

## **Expected Outcomes**

The expected outcomes of this analysis include:

* A clear summary of session delivery volume for each course.
* Insights into training demand trends across the year.
* Recommendations for strategic adjustments in future training plans.

The results will guide planning for the 2026 training calendar, ensuring efficient use of resources and alignment with learner demand.

In [1]:
import pandas as pd

data = pd.read_excel("/kaggle/input/participants2025/TKI-ED-2025-Editions-Delivered-v0.1-CS.xlsx")
# Check Data Structure
print("Data Structure:")
print(data.info())

Data Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4150 entries, 0 to 4149
Data columns (total 51 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Organizer                            4150 non-null   object 
 1   Facilitator                          4150 non-null   object 
 2   Co-Facilitator                       9 non-null      object 
 3   Course title                         4150 non-null   object 
 4   Course structure                     4130 non-null   object 
 5   Method                               4150 non-null   object 
 6   Start date                           4150 non-null   object 
 7   End date                             4150 non-null   object 
 8   Course city                          1985 non-null   object 
 9   Course country                       1986 non-null   object 
 10  Title                                448 non-null    object 
 11  First Name    

### **Data Selection**

In this step, we select only nine columns that are relevant to analysing session delivery in 2025. These columns capture essential course and scheduling information while excluding unrelated or unused fields. This focused selection improves clarity and processing efficiency during analysis.

The selected columns are:

1. **Organizer** – The company or unit responsible for managing the course.
2. **Facilitator** – The instructor leading the session.
3. **Course Title** – The official name of the course.
4. **Course Structure** – The type of session (for example, in-house or external).
5. **Method** – The delivery mode (for example, live or online).
6. **Start Date** – The date when the course begins.
7. **End Date** – The date when the course concludes.
8. **Course City** – The city where the training took place.
9. **Course Country** – The country where the session was delivered.

These columns provide all the information required to measure session delivery, frequency, and geographic distribution throughout the year.

In [2]:
# Select only the required columns for analysis
selected_columns = [
    'Organizer',
    'Facilitator',
    'Course title',
    'Course structure',
    'Method',
    'Start date',
    'End date',
    'Course city',
    'Course country'
]

# Create a new dataset containing only these columns
course = data[selected_columns]

# Display the first few rows to confirm the selection
print("Filtered course dataset (columns 1–9):")
display(course.head())

Filtered course dataset (columns 1–9):


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country
0,Vigilance Consulting,Dr. Saleh Al-Ansari,C-KPI,In-house,Live,19/2/2025,23/2/2025,NaN,Bahrain
1,TKI,Yasser Ghonimy,C-SBP,Open,Live,25/02/2025,29/02/2025,NaN,NaN
2,TKI,Amany Fakhry,C-KPI,Open,Live,19/05/2025,23/05/2025,NaN,NaN
3,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN
4,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN


### Duplicate Check and Participant Count

In this step, we define one session and count how many participant records belong to it. This helps us measure session volume without changing the source data.

#### Session Rule
A session is the unique combination of four columns: **Facilitator**, **Course Title**, **Start Date**, and **End Date**.

#### What We Do
1. Find duplicate rows based on the four columns above.  
2. Add a new column **Participants Count**, which stores the size of each group defined by the four columns.  
3. For any row with no duplicate, set **Participants Count** to 1. This column has no missing values.  
4. Produce three checks for quick validation:  
   - **Total rows**: the number of records in the filtered dataset.  
   - **Total participants count**: the sum of *Participants Count* across unique sessions. This equals the number of rows.  
   - **Total sessions**: the number of unique combinations of the four key columns.  
5. Show a short preview so we can confirm the result.

#### Assumptions and Guardrails
- We do not change the data type of any existing column.  
- We only add the **Participants Count** column.  
- All four grouping columns must be present and readable in the dataset.

In [3]:
# Define the session key
key_cols = ['Facilitator', 'Course title', 'Start date', 'End date']

# Count rows per unique session, aligned back to each row
group_sizes = course.groupby(key_cols, dropna=False).size()
course['Participants count'] = course[key_cols].merge(
    group_sizes.rename('Participants count').reset_index(),
    on=key_cols,
    how='left'
)['Participants count']

# Safety check: ensure no missing values in the new column, default to 1 if any appear
course['Participants count'] = course['Participants count'].fillna(1).astype(int)

# Summary numbers
total_rows = len(course)
total_sessions = group_sizes.shape[0]
total_participants = int(group_sizes.sum())  # equals total_rows by construction

# Print summaries
print("Total rows:", total_rows)
print("Total participants count:", total_participants)
print("Total sessions:", total_sessions)

# Preview
print("\nPreview:")
display(course.head())

Total rows: 4150
Total participants count: 4150
Total sessions: 246

Preview:


/tmp/ipykernel_14/2200101167.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course['Participants count'] = course[key_cols].merge(
/tmp/ipykernel_14/2200101167.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course['Participants count'] = course['Participants count'].fillna(1).astype(int)


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Participants count
0,Vigilance Consulting,Dr. Saleh Al-Ansari,C-KPI,In-house,Live,19/2/2025,23/2/2025,NaN,Bahrain,1
1,TKI,Yasser Ghonimy,C-SBP,Open,Live,25/02/2025,29/02/2025,NaN,NaN,1
2,TKI,Amany Fakhry,C-KPI,Open,Live,19/05/2025,23/05/2025,NaN,NaN,1
3,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN,5
4,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN,5


### Remove duplicates, keep one row per session

We will drop duplicate rows using the session keys, Facilitator, Course title, Start date, End date. We keep the first row of each session. The Participants count column already stores the group size, so the kept row holds the correct count. No other column types change.

In [4]:
# Keys that define one session
key_cols = ['Facilitator', 'Course title', 'Start date', 'End date']

# Remove duplicates, keep the first row per session
course_dedup = course.drop_duplicates(subset=key_cols, keep='first').copy()

# Quick checks
print("Rows before:", len(course))
print("Rows after :", len(course_dedup))
print("Rows removed:", len(course) - len(course_dedup))

print("\nPreview after dedupe:")
display(course_dedup.head())

Rows before: 4150
Rows after : 246
Rows removed: 3904

Preview after dedupe:


,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Participants count
0,Vigilance Consulting,Dr. Saleh Al-Ansari,C-KPI,In-house,Live,19/2/2025,23/2/2025,NaN,Bahrain,1
1,TKI,Yasser Ghonimy,C-SBP,Open,Live,25/02/2025,29/02/2025,NaN,NaN,1
2,TKI,Amany Fakhry,C-KPI,Open,Live,19/05/2025,23/05/2025,NaN,NaN,1
3,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,NaN,NaN,5
8,TKI,Malek Mohammed Ghazo,C-KPI,Open,Live,2025-12-01 00:00:00,16/01/2025,NaN,NaN,24


In [5]:
# Check if there are still duplicates after the cleaning process
remaining_duplicates = course_dedup[course_dedup.duplicated(subset=['Facilitator', 'Course title', 'Start date', 'End date'], keep=False)]

# Display the results
if len(remaining_duplicates) == 0:
    print("✅ No duplicates remain. The data is clean.")
else:
    print(f"⚠️ There are still {len(remaining_duplicates)} duplicates remaining.")
    display(remaining_duplicates.head())

✅ No duplicates remain. The data is clean.


In [6]:
# Configure the table to scroll
def table(df, height=400):
    display(df.style.set_table_attributes(f'style="display:inline-block;overflow:auto;height:{height}px;width:auto;"').set_table_styles([{
        'selector': '',
        'props': [('border-collapse', 'collapse'),
                  ('margin', '0px')]}]))

# Display the table
table(course_dedup)

,Organizer,Facilitator,Course title,Course structure,Method,Start date,End date,Course city,Course country,Participants count
0,Vigilance Consulting,Dr. Saleh Al-Ansari,C-KPI,In-house,Live,19/2/2025,23/2/2025,nan,Bahrain,1
1,TKI,Yasser Ghonimy,C-SBP,Open,Live,25/02/2025,29/02/2025,nan,nan,1
2,TKI,Amany Fakhry,C-KPI,Open,Live,19/05/2025,23/05/2025,nan,nan,1
3,TKI,Radu Cocean,C-SBP,Open,Live,06.01.2025,10.01.2025,nan,nan,5
8,TKI,Malek Mohammed Ghazo,C-KPI,Open,Live,2025-12-01 00:00:00,16/01/2025,nan,nan,24
32,TKI,Raluca Vintila,C-KPI,Open,Live,13/01/2025,17/01/2025,nan,nan,10
42,TKI,Michael Romero,KPI-E,In-house,Face to face,13/01/2025,13/01/2025,Labuan,Malaysia,26
68,Vigilance,Dr. Raed Al-Jowder,C-SBP,Open,Live,12.01.2025,16.01.2025,nan,nan,12
80,TKI,Radu Cocean,"PM, BSC & KPI-M",In-house,Face to face,13/01/2025,14/01/2025,Riyadh,Saudi Arabia,23
103,TKI,Radu Cocean,"PM, BSC & KPI-M",In-house,Face to face,15/01/2025,16/01/2025,Riyadh,Saudi Arabia,33


In [7]:
# Export cleaned data to Excel file
output_path = '/kaggle/working/Course-Season-2025-clean.xlsx'
course_dedup.to_excel(output_path, index=False)

print(f"✅ Data successfully exported to: {output_path}")

✅ Data successfully exported to: /kaggle/working/Course-Season-2025-clean.xlsx


### Standardise and flag dates, handle year first cases

Goal
Create a clean copy from `course_dedup`, convert Start date and End date to text for safe checks, then build standardised dates. Mark any row that can be read in more than one way.

What we will do

1. Copy `course_dedup` to `course_clean`.
2. Create text versions: `Start date raw`, `End date raw`.
3. Normalise the text: trim spaces, replace dot or hyphen with slash.
4. Flag ambiguous text where both the first two tokens are from 1 to 12, for example 12/01/2025.
5. Detect year first strings, for example 2025 12 01 00:00:00. For your data the correct order is year, day, month. We will swap the last two pieces before parsing and mark a flag `Year first swapped`.
6. Parse dates safely in two passes: try day first, then month first for anything still not parsed.
7. Mark a second ambiguity flag when both day first and month first give two different valid dates.
8. Produce a review table that shows both Start and End side by side, with original text, normalised text, the final parsed date in MM/DD/YYYY, and all flags.
9. Print counts.

Notes
We do not change the type of existing columns. We only add new text columns, flags, and final datetime columns.

In [8]:
import numpy as np
import re

# 0) Start from the deduplicated dataset
course_clean = course_dedup.copy()

# 1) Make text copies and normalise separators
course_clean['Start date raw'] = course_clean['Start date'].astype(str)
course_clean['End date raw']   = course_clean['End date'].astype(str)

def normalise_to_slash(s: pd.Series) -> pd.Series:
    """Trim, then replace '-' and '.' with '/'."""
    return s.str.strip().str.replace(r'[-\.]', '/', regex=True)

course_clean['Start date norm'] = normalise_to_slash(course_clean['Start date raw'])
course_clean['End date norm']   = normalise_to_slash(course_clean['End date raw'])

# 2) Helpers
def parse_excel_serial(s_norm: pd.Series) -> pd.Series:
    """Convert plain integer excel serials to datetime."""
    mask = s_norm.str.fullmatch(r'\d+')
    ser = pd.to_numeric(s_norm.where(mask), errors='coerce')
    return pd.to_datetime(ser, unit='d', origin='1899-12-30', errors='coerce')

def looks_year_first(txt: str) -> bool:
    """True if string starts with 4 digits then a separator: YYYY/..."""
    return bool(re.match(r'^\s*\d{4}[/\-\.\s]', str(txt)))

def swap_year_first_to_yyyy_dd_mm(s_norm: pd.Series) -> pd.Series:
    """
    Your rule: when string looks year-first and both middle tokens are 1..12,
    interpret as YYYY/DD/MM, not YYYY/MM/DD.
    We swap the last two tokens in that case, and keep any time part.
    """
    def swapper(x: str) -> str:
        raw = str(x).strip()
        if not looks_year_first(raw):
            return raw
        t = re.sub(r'[-\.]', '/', raw)  # normalise
        # separate time from date
        if ' ' in t:
            date_part, time_part = t.split(' ', 1)
        else:
            date_part, time_part = t, None
        parts = date_part.split('/')
        if len(parts) >= 3 and parts[1].isdigit() and parts[2].isdigit():
            a, b = int(parts[1]), int(parts[2])
            if 1 <= a <= 12 and 1 <= b <= 12:
                parts[1], parts[2] = parts[2], parts[1]  # swap month and day
                date_swapped = '/'.join(parts)
                return date_swapped + (f' {time_part}' if time_part else '')
        return raw
    return s_norm.map(swapper)

def ambiguous_text_flag(txt: str) -> bool:
    """
    Ambiguous text if after normalising, both the first two tokens are 1..12:
    e.g., 12/01/2025 can be day-first or month-first.
    """
    t = str(txt).strip()
    t = re.sub(r'[-\.]', '/', t)
    parts = t.split('/')
    if len(parts) >= 2:
        try:
            a, b = int(parts[0]), int(parts[1])
            return 1 <= a <= 12 and 1 <= b <= 12
        except ValueError:
            return False
    return False

def parser_disagreement(s: pd.Series) -> pd.Series:
    """
    True if parsing the same text day-first and month-first both succeed
    and yield different dates. That means the text is ambiguous.
    """
    p_day = pd.to_datetime(s, dayfirst=True, errors='coerce')
    p_mon = pd.to_datetime(s, dayfirst=False, errors='coerce')
    both_ok = p_day.notna() & p_mon.notna()
    return both_ok & (p_day != p_mon)

def parse_two_pass_with_swap(s_norm: pd.Series, s_swapped: pd.Series) -> pd.Series:
    """
    Parse with priority:
      1) Excel serial
      2) Day-first on swapped text
      3) Month-first on swapped text where 2) failed
    """
    parsed_serial = parse_excel_serial(s_norm)
    p1 = pd.to_datetime(s_swapped, dayfirst=True, errors='coerce')
    need_second = p1.isna()
    p2 = pd.to_datetime(s_swapped.where(need_second), dayfirst=False, errors='coerce')
    return parsed_serial.combine_first(p1).combine_first(p2)

def to_mmddyyyy(s: pd.Series) -> pd.Series:
    """Format datetimes for human review only."""
    return s.dt.strftime('%m/%d/%Y')

# 3) Year-first swap text and flags
course_clean['Start date swapped'] = swap_year_first_to_yyyy_dd_mm(course_clean['Start date norm'])
course_clean['End date swapped']   = swap_year_first_to_yyyy_dd_mm(course_clean['End date norm'])

course_clean['Start year first swapped'] = course_clean['Start date swapped'].ne(course_clean['Start date norm'])
course_clean['End year first swapped']   = course_clean['End date swapped'].ne(course_clean['End date norm'])

# 4) Parse to standard datetime columns (keep originals unchanged)
course_clean['Start date std'] = parse_two_pass_with_swap(course_clean['Start date norm'],
                                                          course_clean['Start date swapped'])
course_clean['End date std']   = parse_two_pass_with_swap(course_clean['End date norm'],
                                                          course_clean['End date swapped'])

# 5) Ambiguity flags
# Text ambiguity
course_clean['Start amb text'] = course_clean['Start date norm'].map(ambiguous_text_flag)
course_clean['End amb text']   = course_clean['End date norm'].map(ambiguous_text_flag)

# Parser disagreement on normalised text (without swap)
course_clean['Start amb parsed'] = parser_disagreement(course_clean['Start date norm'])
course_clean['End amb parsed']   = parser_disagreement(course_clean['End date norm'])

# Combined ambiguity flag
course_clean['Start ambiguous'] = course_clean[['Start amb text',
                                                'Start year first swapped',
                                                'Start amb parsed']].any(axis=1)
course_clean['End ambiguous']   = course_clean[['End amb text',
                                                'End year first swapped',
                                                'End amb parsed']].any(axis=1)

# 6) Build human-readable strings for review
course_clean['Start date std (MM/DD/YYYY)'] = to_mmddyyyy(course_clean['Start date std'])
course_clean['End date std (MM/DD/YYYY)']   = to_mmddyyyy(course_clean['End date std'])

# 7) Review tables and prints
# Compact review table
review_cols = [
    'Organizer', 'Facilitator', 'Course title',
    'Start date raw', 'Start date norm', 'Start date swapped', 'Start date std',
    'End date raw',   'End date norm',   'End date swapped',   'End date std',
    'Start ambiguous', 'End ambiguous'
]
review_table = course_clean[review_cols].copy()
review_table['Start date std (MM/DD/YYYY)'] = course_clean['Start date std (MM/DD/YYYY)']
review_table['End date std (MM/DD/YYYY)']   = course_clean['End date std (MM/DD/YYYY)']

# Ambiguous rows table, show both start and end for easy review
amb_table = review_table[
    review_table['Start ambiguous'] | review_table['End ambiguous']
][[
    'Organizer', 'Facilitator', 'Course title',
    'Start date raw', 'Start date norm', 'Start date swapped', 'Start date std (MM/DD/YYYY)',
    'End date raw',   'End date norm',   'End date swapped',   'End date std (MM/DD/YYYY)',
    'Start ambiguous', 'End ambiguous'
]].copy()

# NaT rows table: where parsing failed on start or end
nat_mask = course_clean['Start date std'].isna() | course_clean['End date std'].isna()
nat_table = review_table[nat_mask][[
    'Organizer', 'Facilitator', 'Course title',
    'Start date raw', 'Start date norm', 'Start date swapped', 'Start date std',
    'End date raw',   'End date norm',   'End date swapped',   'End date std'
]].copy()

# Summary counts
rows_total = len(course_clean)
start_ok = course_clean['Start date std'].notna().sum()
end_ok   = course_clean['End date std'].notna().sum()
amb_count = len(amb_table)
nat_count = len(nat_table)

print("Rows:", rows_total)
print("Parsed start:", start_ok, "| NaT start:", rows_total - start_ok)
print("Parsed end  :", end_ok,   "| NaT end  :", rows_total - end_ok)
print("Ambiguous rows:", amb_count)
print("NaT rows     :", nat_count)

# Show quick previews for your manual review
print("\nHead of AMBIGUOUS table:")
display(amb_table.head(20))

print("\nHead of NaT table:")
display(nat_table.head(20))

Rows: 246
Parsed start: 246 | NaT start: 0
Parsed end  : 244 | NaT end  : 2
Ambiguous rows: 107
NaT rows     : 2

Head of AMBIGUOUS table:


/tmp/ipykernel_14/1070497572.py:91: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  p2 = pd.to_datetime(s_swapped.where(need_second), dayfirst=False, errors='coerce')
/tmp/ipykernel_14/1070497572.py:77: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  p_mon = pd.to_datetime(s, dayfirst=False, errors='coerce')
/tmp/ipykernel_14/1070497572.py:77: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  p_mon = pd.to_datetime(s, dayfirst=False, errors='coerce')


,Organizer,Facilitator,Course title,Start date raw,Start date norm,Start date swapped,Start date std (MM/DD/YYYY),End date raw,End date norm,End date swapped,End date std (MM/DD/YYYY),Start ambiguous,End ambiguous
3,TKI,Radu Cocean,C-SBP,06.01.2025,06/01/2025,06/01/2025,01/06/2025,10.01.2025,10/01/2025,10/01/2025,01/10/2025,True,True
8,TKI,Malek Mohammed Ghazo,C-KPI,2025-12-01 00:00:00,2025/12/01 00:00:00,2025/01/12 00:00:00,01/12/2025,16/01/2025,16/01/2025,16/01/2025,01/16/2025,True,False
68,Vigilance,Dr. Raed Al-Jowder,C-SBP,12.01.2025,12/01/2025,12/01/2025,01/12/2025,16.01.2025,16/01/2025,16/01/2025,01/16/2025,True,False
381,TKI,Dr. Alina Miertoiu,C-KPIPP,2025-02-02 00:00:00,2025/02/02 00:00:00,2025/02/02 00:00:00,02/02/2025,2025-02-06 00:00:00,2025/02/06 00:00:00,2025/06/02 00:00:00,06/02/2025,False,True
397,TKI,Andrea Minelli,C-BSC,2025-02-02 00:00:00,2025/02/02 00:00:00,2025/02/02 00:00:00,02/02/2025,2025-06-02 00:00:00,2025/06/02 00:00:00,2025/02/06 00:00:00,02/06/2025,False,True
406,TKI,Cristina Mihailoaie,C-PA,2025-03-02 00:00:00,2025/03/02 00:00:00,2025/02/03 00:00:00,02/03/2025,2025-07-02 00:00:00,2025/07/02 00:00:00,2025/02/07 00:00:00,02/07/2025,True,True
413,TKI,Mihai Toma,C-BSC,2025-03-02 00:00:00,2025/03/02 00:00:00,2025/02/03 00:00:00,02/03/2025,2025-07-02 00:00:00,2025/07/02 00:00:00,2025/02/07 00:00:00,02/07/2025,True,True
419,TKI,Teodora Gorski,C-KPI,3.02.2025,3/02/2025,3/02/2025,02/03/2025,7.02.2025,7/02/2025,7/02/2025,02/07/2025,True,True
431,TKI,Raluca Vintila,KPI-E,2025-05-02 00:00:00,2025/05/02 00:00:00,2025/02/05 00:00:00,02/05/2025,2025-05-02 00:00:00,2025/05/02 00:00:00,2025/02/05 00:00:00,02/05/2025,True,True
458,PwC,Abdulaziz Hussein,C-SBP,02.02.2025,02/02/2025,02/02/2025,02/02/2025,04.02.2025,04/02/2025,04/02/2025,02/04/2025,True,True



Head of NaT table:


,Organizer,Facilitator,Course title,Start date raw,Start date norm,Start date swapped,Start date std,End date raw,End date norm,End date swapped,End date std
1,TKI,Yasser Ghonimy,C-SBP,25/02/2025,25/02/2025,25/02/2025,2025-02-25,29/02/2025,29/02/2025,29/02/2025,NaT
2338,TKI,Yassine Chaker,C-B,22/6/2025,22/6/2025,22/6/2025,2025-06-22,26/6/20225,26/6/20225,26/6/20225,NaT


## Date Review and Fix Plan

### Scope
* Keep all original date columns unchanged.
* Work only on **Start Date Std** and **End Date Std**.

### Step 1, resolve NaT rows
**What we will do**
1. List rows with NaT in standardised columns.
2. Enter corrected dates in a dictionary.
3. Update only **Start Date Std** and **End Date Std**.

Rows 1 and 2338 will be fixed:  
* Row 1 → End Date changed to **01/03/2025**  Note: 29 Feb 2025 is invalid, replaced with 01 Mar 2025.
* Row 2338 → End Date changed to **26/06/2025**

### Step 2, resolve Ambiguous rows
**What we will do**

1. Swap month and day for the rows you listed.

   * End date wrong at rows: 381, 485, 2203, 2233, 2686, 2776, 3184, 3239.
   * Start date wrong at rows: 695, 971, 1743, 2233, 3184, 3239, 3348.
2. Keep all other columns and types unchanged.
3. Print a quick check. Show the edited rows with both original text and the fixed datetime.
4. Report any rows where End date is earlier than Start date and show a small table for review.

   * Note: row 1772 is expected to appear here because End date is earlier than Start date.

### Guarantees
* No other column type is changed.
* Only **Start Date Std** and **End Date Std** are updated.

### 1) Fix NaT rows

In [9]:
# Before: check current NaT rows
def _build_nat_table(df):
    mask = df['Start date std'].isna() | df['End date std'].isna()
    cols = [
        'Organizer','Facilitator','Course title',
        'Start date raw','Start date norm','Start date swapped','Start date std',
        'End date raw','End date norm','End date swapped','End date std'
    ]
    t = df.loc[mask, cols].copy()
    return t

nat_before = _build_nat_table(course_clean)
print("NaT rows BEFORE fix:", len(nat_before))
display(nat_before.head(20))

# Apply targeted NaT fixes you provided
# {"idx": 1, "start": None, "end": "2025-03-01"}   from 29/02/2025 -> 2025-03-01
# {"idx": 2338, "start": None, "end": "2025-06-26"} from 26/6/20225 -> 2025-06-26
nat_fixes = [
    {"idx": 1,    "start": None,         "end": "2025-03-01"},
    {"idx": 2338, "start": None,         "end": "2025-06-26"},
]

for fix in nat_fixes:
    i = fix["idx"]
    if fix.get("start") is not None:
        course_clean.at[i, 'Start date std'] = pd.to_datetime(fix["start"], errors='coerce')
        course_clean.at[i, 'Start date std (MM/DD/YYYY)'] = course_clean.at[i, 'Start date std'].strftime('%m/%d/%Y') \
            if pd.notna(course_clean.at[i, 'Start date std']) else None
    if fix.get("end") is not None:
        course_clean.at[i, 'End date std'] = pd.to_datetime(fix["end"], errors='coerce')
        course_clean.at[i, 'End date std (MM/DD/YYYY)'] = course_clean.at[i, 'End date std'].strftime('%m/%d/%Y') \
            if pd.notna(course_clean.at[i, 'End date std']) else None

# After: check NaT rows again
nat_after = _build_nat_table(course_clean)
print("NaT rows AFTER fix:", len(nat_after))
display(nat_after.head(20))

NaT rows BEFORE fix: 2


,Organizer,Facilitator,Course title,Start date raw,Start date norm,Start date swapped,Start date std,End date raw,End date norm,End date swapped,End date std
1,TKI,Yasser Ghonimy,C-SBP,25/02/2025,25/02/2025,25/02/2025,2025-02-25,29/02/2025,29/02/2025,29/02/2025,NaT
2338,TKI,Yassine Chaker,C-B,22/6/2025,22/6/2025,22/6/2025,2025-06-22,26/6/20225,26/6/20225,26/6/20225,NaT


NaT rows AFTER fix: 0


,Organizer,Facilitator,Course title,Start date raw,Start date norm,Start date swapped,Start date std,End date raw,End date norm,End date swapped,End date std


### 2) Fix Ambiguous rows

In [10]:
# Helper to swap month and day of a Timestamp safely
def _swap_month_day(ts):
    if pd.isna(ts):
        return ts
    try:
        return pd.Timestamp(year=ts.year, month=ts.day, day=ts.month)
    except ValueError:
        # If swap is invalid, return original so you can review later
        return ts

# Before: build ambiguous review table
def _build_amb_table(df):
    cols = [
        'Organizer','Facilitator','Course title',
        'Start date raw','Start date norm','Start date swapped','Start date std',
        'End date raw','End date norm','End date swapped','End date std',
        'Start ambiguous','End ambiguous',
        'Start date std (MM/DD/YYYY)','End date std (MM/DD/YYYY)'
    ]
    t = df.loc[df['Start ambiguous'] | df['End ambiguous'], cols].copy()
    return t

amb_before = _build_amb_table(course_clean)
print("Ambiguous rows BEFORE fix:", len(amb_before))
# Configure the table to scroll
def table(df, height=400):
    display(df.style.set_table_attributes(f'style="display:inline-block;overflow:auto;height:{height}px;width:auto;"').set_table_styles([{
        'selector': '',
        'props': [('border-collapse', 'collapse'),
                  ('margin', '0px')]}]))

# Display the table
table(amb_before)

# Indices provided
end_wrong_idx   = [381, 485, 2203, 2233, 2686, 2776, 3184, 3239]
start_wrong_idx = [695, 971, 1743, 2233, 3184, 3239, 3348]

# Fix start side
for i in start_wrong_idx:
    ts = course_clean.at[i, 'Start date std']
    ts_new = _swap_month_day(ts)
    course_clean.at[i, 'Start date std'] = ts_new
    course_clean.at[i, 'Start date std (MM/DD/YYYY)'] = ts_new.strftime('%m/%d/%Y') if pd.notna(ts_new) else None
    # Mark resolved on start side
    course_clean.at[i, 'Start ambiguous'] = False

# Fix end side
for i in end_wrong_idx:
    ts = course_clean.at[i, 'End date std']
    ts_new = _swap_month_day(ts)
    course_clean.at[i, 'End date std'] = ts_new
    course_clean.at[i, 'End date std (MM/DD/YYYY)'] = ts_new.strftime('%m/%d/%Y') if pd.notna(ts_new) else None
    # Mark resolved on end side
    course_clean.at[i, 'End ambiguous'] = False

# After: show ambiguous rows again
amb_after = _build_amb_table(course_clean)
print("Ambiguous rows AFTER fix:", len(amb_after))
# Configure the table to scroll
def table(df, height=400):
    display(df.style.set_table_attributes(f'style="display:inline-block;overflow:auto;height:{height}px;width:auto;"').set_table_styles([{
        'selector': '',
        'props': [('border-collapse', 'collapse'),
                  ('margin', '0px')]}]))

# Display the table
table(amb_after)

Ambiguous rows BEFORE fix: 107


,Organizer,Facilitator,Course title,Start date raw,Start date norm,Start date swapped,Start date std,End date raw,End date norm,End date swapped,End date std,Start ambiguous,End ambiguous,Start date std (MM/DD/YYYY),End date std (MM/DD/YYYY)
3,TKI,Radu Cocean,C-SBP,06.01.2025,06/01/2025,06/01/2025,2025-01-06 00:00:00,10.01.2025,10/01/2025,10/01/2025,2025-01-10 00:00:00,True,True,01/06/2025,01/10/2025
8,TKI,Malek Mohammed Ghazo,C-KPI,2025-12-01 00:00:00,2025/12/01 00:00:00,2025/01/12 00:00:00,2025-01-12 00:00:00,16/01/2025,16/01/2025,16/01/2025,2025-01-16 00:00:00,True,False,01/12/2025,01/16/2025
68,Vigilance,Dr. Raed Al-Jowder,C-SBP,12.01.2025,12/01/2025,12/01/2025,2025-01-12 00:00:00,16.01.2025,16/01/2025,16/01/2025,2025-01-16 00:00:00,True,False,01/12/2025,01/16/2025
381,TKI,Dr. Alina Miertoiu,C-KPIPP,2025-02-02 00:00:00,2025/02/02 00:00:00,2025/02/02 00:00:00,2025-02-02 00:00:00,2025-02-06 00:00:00,2025/02/06 00:00:00,2025/06/02 00:00:00,2025-06-02 00:00:00,False,True,02/02/2025,06/02/2025
397,TKI,Andrea Minelli,C-BSC,2025-02-02 00:00:00,2025/02/02 00:00:00,2025/02/02 00:00:00,2025-02-02 00:00:00,2025-06-02 00:00:00,2025/06/02 00:00:00,2025/02/06 00:00:00,2025-02-06 00:00:00,False,True,02/02/2025,02/06/2025
406,TKI,Cristina Mihailoaie,C-PA,2025-03-02 00:00:00,2025/03/02 00:00:00,2025/02/03 00:00:00,2025-02-03 00:00:00,2025-07-02 00:00:00,2025/07/02 00:00:00,2025/02/07 00:00:00,2025-02-07 00:00:00,True,True,02/03/2025,02/07/2025
413,TKI,Mihai Toma,C-BSC,2025-03-02 00:00:00,2025/03/02 00:00:00,2025/02/03 00:00:00,2025-02-03 00:00:00,2025-07-02 00:00:00,2025/07/02 00:00:00,2025/02/07 00:00:00,2025-02-07 00:00:00,True,True,02/03/2025,02/07/2025
419,TKI,Teodora Gorski,C-KPI,3.02.2025,3/02/2025,3/02/2025,2025-02-03 00:00:00,7.02.2025,7/02/2025,7/02/2025,2025-02-07 00:00:00,True,True,02/03/2025,02/07/2025
431,TKI,Raluca Vintila,KPI-E,2025-05-02 00:00:00,2025/05/02 00:00:00,2025/02/05 00:00:00,2025-02-05 00:00:00,2025-05-02 00:00:00,2025/05/02 00:00:00,2025/02/05 00:00:00,2025-02-05 00:00:00,True,True,02/05/2025,02/05/2025
458,PwC,Abdulaziz Hussein,C-SBP,02.02.2025,02/02/2025,02/02/2025,2025-02-02 00:00:00,04.02.2025,04/02/2025,04/02/2025,2025-02-04 00:00:00,True,True,02/02/2025,02/04/2025


Ambiguous rows AFTER fix: 95


,Organizer,Facilitator,Course title,Start date raw,Start date norm,Start date swapped,Start date std,End date raw,End date norm,End date swapped,End date std,Start ambiguous,End ambiguous,Start date std (MM/DD/YYYY),End date std (MM/DD/YYYY)
3,TKI,Radu Cocean,C-SBP,06.01.2025,06/01/2025,06/01/2025,2025-01-06 00:00:00,10.01.2025,10/01/2025,10/01/2025,2025-01-10 00:00:00,True,True,01/06/2025,01/10/2025
8,TKI,Malek Mohammed Ghazo,C-KPI,2025-12-01 00:00:00,2025/12/01 00:00:00,2025/01/12 00:00:00,2025-01-12 00:00:00,16/01/2025,16/01/2025,16/01/2025,2025-01-16 00:00:00,True,False,01/12/2025,01/16/2025
68,Vigilance,Dr. Raed Al-Jowder,C-SBP,12.01.2025,12/01/2025,12/01/2025,2025-01-12 00:00:00,16.01.2025,16/01/2025,16/01/2025,2025-01-16 00:00:00,True,False,01/12/2025,01/16/2025
397,TKI,Andrea Minelli,C-BSC,2025-02-02 00:00:00,2025/02/02 00:00:00,2025/02/02 00:00:00,2025-02-02 00:00:00,2025-06-02 00:00:00,2025/06/02 00:00:00,2025/02/06 00:00:00,2025-02-06 00:00:00,False,True,02/02/2025,02/06/2025
406,TKI,Cristina Mihailoaie,C-PA,2025-03-02 00:00:00,2025/03/02 00:00:00,2025/02/03 00:00:00,2025-02-03 00:00:00,2025-07-02 00:00:00,2025/07/02 00:00:00,2025/02/07 00:00:00,2025-02-07 00:00:00,True,True,02/03/2025,02/07/2025
413,TKI,Mihai Toma,C-BSC,2025-03-02 00:00:00,2025/03/02 00:00:00,2025/02/03 00:00:00,2025-02-03 00:00:00,2025-07-02 00:00:00,2025/07/02 00:00:00,2025/02/07 00:00:00,2025-02-07 00:00:00,True,True,02/03/2025,02/07/2025
419,TKI,Teodora Gorski,C-KPI,3.02.2025,3/02/2025,3/02/2025,2025-02-03 00:00:00,7.02.2025,7/02/2025,7/02/2025,2025-02-07 00:00:00,True,True,02/03/2025,02/07/2025
431,TKI,Raluca Vintila,KPI-E,2025-05-02 00:00:00,2025/05/02 00:00:00,2025/02/05 00:00:00,2025-02-05 00:00:00,2025-05-02 00:00:00,2025/05/02 00:00:00,2025/02/05 00:00:00,2025-02-05 00:00:00,True,True,02/05/2025,02/05/2025
458,PwC,Abdulaziz Hussein,C-SBP,02.02.2025,02/02/2025,02/02/2025,2025-02-02 00:00:00,04.02.2025,04/02/2025,04/02/2025,2025-02-04 00:00:00,True,True,02/02/2025,02/04/2025
494,TKI,Mohamed Khaled,C-TP,2025-02-02 00:00:00,2025/02/02 00:00:00,2025/02/02 00:00:00,2025-02-02 00:00:00,2025-06-02 00:00:00,2025/06/02 00:00:00,2025/02/06 00:00:00,2025-02-06 00:00:00,False,True,02/02/2025,02/06/2025


## Add Full Course Name and Summarise 2025 Sessions

### What will be done
1. Read the grouped sheet from the Excel file at `/kaggle/input/participants2025/TKI-ED-2025-Editions-Delivered-v0.1-CS.xlsx`.  
2. Map **Course Title** acronyms to full names and add a new column, **Course Name**.  
3. Filter sessions to calendar year **2025**.  
4. Print totals for **sessions** and **participants**.  
5. Build quick summaries by **Course Name**, **Facilitator**, **Month**, **Method**, and **Country**.  
6. Show short head previews for each summary.  
7. Optionally export all outputs to a single Excel file.

### Outputs
* A dataset with an added **Course Name** column.  
* Counts for total sessions and total participants in 2025.  
* Summary tables by course, facilitator, month, method, and country.  
* Head previews for quick checks.  
* One optional Excel export that contains all results.

### Assumptions and guardrails
* No existing column types are changed.  
* Only the **Course Name** column is added.  
* The mapping for acronyms to full names is complete and correct, or any missing items are reported.  
* Year filtering uses the start date of each session.  

In [11]:
import pandas as pd

# ============================================================
# 0) Preconditions and helpers
#    - Assumes you already have course_dedup (with Participants count)
#    - If dates live in course_clean, we copy them in
# ============================================================

# If standardised dates exist in course_clean, bring them into course_dedup for this step
if 'course_clean' in globals():
    for col in [
        'Start date std','End date std',
        'Start date std (MM/DD/YYYY)','End date std (MM/DD/YYYY)'
    ]:
        if col in course_clean.columns:
            course_dedup[col] = course_clean[col]

# Safety check: readable date columns for review
def _ensure_readable_dates(df):
    if 'Start date std (MM/DD/YYYY)' not in df.columns and 'Start date std' in df.columns:
        df['Start date std (MM/DD/YYYY)'] = df['Start date std'].dt.strftime('%m/%d/%Y')
    if 'End date std (MM/DD/YYYY)' not in df.columns and 'End date std' in df.columns:
        df['End date std (MM/DD/YYYY)'] = df['End date std'].dt.strftime('%m/%d/%Y')
_ensure_readable_dates(course_dedup)

# ============================================================
# 1) Read mapping from the 'grouped' sheet (no merge, build dict)
# ============================================================
mapping_path = "/kaggle/input/participants2025/TKI-ED-2025-Editions-Delivered-v0.1-CS.xlsx"
grouped_map = pd.read_excel(mapping_path, sheet_name="grouped")

# Build a clean dict: ACRONYM -> Full name
map_df = (
    grouped_map[['Acronym', 'Product Name']]
    .dropna(subset=['Acronym'])
    .drop_duplicates(subset=['Acronym'])
    .rename(columns={'Product Name': 'Course Name'})
)
map_df['acronym_key'] = map_df['Acronym'].astype(str).str.strip().str.upper()
name_dict = dict(zip(map_df['acronym_key'], map_df['Course Name']))

# Create a join key on course_dedup
acronym_key = course_dedup['Course title'].astype(str).str.strip().str.upper()

# Safely set or overwrite 'Course Name' using the dict, fallback to original Course title
course_dedup['Course Name'] = acronym_key.map(name_dict).fillna(course_dedup['Course title'])

# Quick preview of mapping results
print("Preview of Course title -> Course Name:")
display(course_dedup[['Course title', 'Course Name']].drop_duplicates().head(20))

# ============================================================
# 2) Filter to sessions in calendar year 2025 using Start date std
# ============================================================
if 'Start date std' not in course_dedup.columns:
    raise ValueError("Missing 'Start date std'. Please run the date standardisation step first.")

data_2025 = course_dedup[course_dedup['Start date std'].dt.year == 2025].copy()

# Keep readable date columns for checking
_ensure_readable_dates(data_2025)

# ============================================================
# 3) Totals (sessions and participants)
# ============================================================
total_sessions_2025 = len(data_2025)
total_participants_2025 = int(data_2025['Participants count'].sum())

print("Total sessions 2025:", total_sessions_2025)
print("Total participants 2025:", total_participants_2025)

# ============================================================
# 4) Summaries
# ============================================================

# By course (use full Course Name)
by_course = (
    data_2025
    .groupby('Course Name', as_index=False)
    .agg(Sessions=('Course Name', 'size'),
         Participants=('Participants count', 'sum'))
    .sort_values(['Sessions', 'Participants'], ascending=False)
)

# By facilitator
by_facilitator = (
    data_2025
    .groupby('Facilitator', as_index=False)
    .agg(Sessions=('Facilitator', 'size'),
         Participants=('Participants count', 'sum'))
    .sort_values(['Sessions', 'Participants'], ascending=False)
)

# By month, derived from Start date std
data_2025['Month'] = data_2025['Start date std'].dt.to_period('M').dt.to_timestamp()
by_month = (
    data_2025
    .groupby('Month', as_index=False)
    .agg(Sessions=('Month', 'size'),
         Participants=('Participants count', 'sum'))
    .sort_values('Month')
)

# By method
by_method = (
    data_2025
    .groupby('Method', as_index=False)
    .agg(Sessions=('Method', 'size'),
         Participants=('Participants count', 'sum'))
    .sort_values(['Sessions', 'Participants'], ascending=False)
)

# By country
by_country = (
    data_2025
    .groupby('Course country', as_index=False)
    .agg(Sessions=('Course country', 'size'),
         Participants=('Participants count', 'sum'))
    .sort_values(['Sessions', 'Participants'], ascending=False)
)

# ============================================================
# 5) Previews with readable date columns included
# ============================================================
cols_preview = [
    'Organizer','Facilitator','Course title','Course Name','Method','Course city','Course country',
    'Start date std','Start date std (MM/DD/YYYY)',
    'End date std','End date std (MM/DD/YYYY)',
    'Participants count'
]
print("\nPreview, data_2025 (with readable dates):")
display(data_2025[cols_preview].head(20))

print("\nSessions by course (full name):")
display(by_course.head(20))

print("\nSessions by facilitator:")
display(by_facilitator.head(20))

print("\nSessions by month:")
display(by_month.head(20))

print("\nSessions by method:")
display(by_method.head(20))

print("\nSessions by country:")
display(by_country.head(20))

# ============================================================
# 6) Optional export of all outputs to a single Excel file
# ============================================================
EXPORT_TO_EXCEL = False  # set to False if you do not want a file
OUTPUT_PATH = "/kaggle/working/Course-Season-2025-summaries.xlsx"

if EXPORT_TO_EXCEL:
    with pd.ExcelWriter(OUTPUT_PATH, engine='xlsxwriter') as xw:
        # Detail sheet keeps readable date columns
        data_2025.to_excel(xw, index=False, sheet_name='Detail_2025')
        by_course.to_excel(xw, index=False, sheet_name='By_Course')
        by_facilitator.to_excel(xw, index=False, sheet_name='By_Facilitator')
        by_month.to_excel(xw, index=False, sheet_name='By_Month')
        by_method.to_excel(xw, index=False, sheet_name='By_Method')
        by_country.to_excel(xw, index=False, sheet_name='By_Country')
    print(f"\nSaved to: {OUTPUT_PATH}")

Preview of Course title -> Course Name:


,Course title,Course Name
0,C-KPI,Certified KPI Professional
1,C-SBP,Certified Strategy and Business Planning Profe...
42,KPI-E,KPI Essentials
80,"PM, BSC & KPI-M","PM, BSC & KPI-M"
136,"PM, BSC, & KPI-Awareness","PM, BSC, & KPI-Awareness"
157,"PM,BSC & KPI Masterclass","PM,BSC & KPI Masterclass"
223,C-PM,Certified Performance Management Professional
242,C-BSC,Certified Balanced Scorecard Management System...
326,C-KPIPP,Certified KPI Professional and Practitioner
406,C-PA,C-PA


Total sessions 2025: 246
Total participants 2025: 4150

Preview, data_2025 (with readable dates):


,Organizer,Facilitator,Course title,Course Name,Method,Course city,Course country,Start date std,Start date std (MM/DD/YYYY),End date std,End date std (MM/DD/YYYY),Participants count
0,Vigilance Consulting,Dr. Saleh Al-Ansari,C-KPI,Certified KPI Professional,Live,NaN,Bahrain,2025-02-19,02/19/2025,2025-02-23,02/23/2025,1
1,TKI,Yasser Ghonimy,C-SBP,Certified Strategy and Business Planning Profe...,Live,NaN,NaN,2025-02-25,02/25/2025,2025-03-01,03/01/2025,1
2,TKI,Amany Fakhry,C-KPI,Certified KPI Professional,Live,NaN,NaN,2025-05-19,05/19/2025,2025-05-23,05/23/2025,1
3,TKI,Radu Cocean,C-SBP,Certified Strategy and Business Planning Profe...,Live,NaN,NaN,2025-01-06,01/06/2025,2025-01-10,01/10/2025,5
8,TKI,Malek Mohammed Ghazo,C-KPI,Certified KPI Professional,Live,NaN,NaN,2025-01-12,01/12/2025,2025-01-16,01/16/2025,24
32,TKI,Raluca Vintila,C-KPI,Certified KPI Professional,Live,NaN,NaN,2025-01-13,01/13/2025,2025-01-17,01/17/2025,10
42,TKI,Michael Romero,KPI-E,KPI Essentials,Face to face,Labuan,Malaysia,2025-01-13,01/13/2025,2025-01-13,01/13/2025,26
68,Vigilance,Dr. Raed Al-Jowder,C-SBP,Certified Strategy and Business Planning Profe...,Live,NaN,NaN,2025-01-12,01/12/2025,2025-01-16,01/16/2025,12
80,TKI,Radu Cocean,"PM, BSC & KPI-M","PM, BSC & KPI-M",Face to face,Riyadh,Saudi Arabia,2025-01-13,01/13/2025,2025-01-14,01/14/2025,23
103,TKI,Radu Cocean,"PM, BSC & KPI-M","PM, BSC & KPI-M",Face to face,Riyadh,Saudi Arabia,2025-01-15,01/15/2025,2025-01-16,01/16/2025,33



Sessions by course (full name):


...

,Month,Sessions,Participants
0,2025-01-01,18,378
1,2025-02-01,34,480
2,2025-03-01,14,249
3,2025-04-01,26,529
4,2025-05-01,37,558
5,2025-06-01,28,509
6,2025-07-01,25,410
7,2025-08-01,38,601
8,2025-09-01,26,436



Sessions by method:


,Method,Sessions,Participants
1,Live,136,2134
0,Face to face,110,2016



Sessions by country:


,Course country,Sessions,Participants
8,Saudi Arabia,76,1517
10,United Arab Emirates,10,106
6,Malaysia,7,86
11,United Kingdom,4,31
1,Brunei Darussalam,3,55
2,Cambodia,2,121
5,Kuwait,2,14
9,Thailand,1,33
7,Qatar,1,10
4,Jordan,1,7
